In [1]:
import pandas as pd

# Ler o arquivo Excel
df = pd.read_excel('systematik.xlsx')

# Exibir os dados para verificar
print(df.head())

              Root         Branch Sub-Branch        Leaf Subleaf  URL
0  Einzelne Länder         Afrika        NaN         NaN     NaN  NaN
1  Einzelne Länder          Asien      Japan   OA/Jap: I     NaN  NaN
2  Einzelne Länder          Asien      Japan  OA/Jap: II     NaN  NaN
3  Einzelne Länder     Australien        NaN         NaN     NaN  NaN
4  Einzelne Länder  Mittelamerika        NaN         NaN     NaN  NaN


In [7]:
class Node:
    def __init__(self, name, url=None):
        self.name = name
        self.url = url
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

def build_tree_from_df(df):
    roots = {}
    for _, row in df.iterrows():
        root_name = row['Root']
        branch_name = row['Branch']
        sub_branch_name = row['Sub-Branch']
        leaf_name = row['Leaf']
        subleaf_name = row['Subleaf']
        url = row['URL']

        if root_name not in roots:
            roots[root_name] = Node(root_name)
        root = roots[root_name]

        branch = next((child for child in root.children if child.name == branch_name), None)
        if not branch:
            branch = Node(branch_name)
            root.add_child(branch)

        if pd.notna(sub_branch_name):
            sub_branch = next((child for child in branch.children if child.name == sub_branch_name), None)
            if not sub_branch:
                sub_branch = Node(sub_branch_name)
                branch.add_child(sub_branch)
        else:
            sub_branch = None

        if pd.notna(leaf_name):
            if sub_branch:
                leaf = next((child for child in sub_branch.children if child.name == leaf_name), None)
                if not leaf:
                    leaf = Node(leaf_name)
                    sub_branch.add_child(leaf)
            else:
                leaf = next((child for child in branch.children if child.name == leaf_name), None)
                if not leaf:
                    leaf = Node(leaf_name)
                    branch.add_child(leaf)
        else:
            leaf = None

        if pd.notna(subleaf_name):
            if leaf:
                subleaf = Node(subleaf_name, url)
                leaf.add_child(subleaf)
        elif leaf and pd.notna(url):
            leaf.url = url
        elif sub_branch and pd.notna(url):
            sub_branch.url = url
        elif branch and pd.notna(url):
            branch.url = url

    return list(roots.values())


In [8]:
def generate_html(node):
    if node.url:
        name_html = f'<a href="{node.url}" target="_blank">{node.name}</a>'
    else:
        name_html = node.name

    if not node.children:
        return f'<li>{name_html}</li>'
    
    children_html = ''.join(generate_html(child) for child in node.children)
    return f'''
    <li>
        <span class="caret">{name_html}</span>
        <ul class="nested">
            {children_html}
        </ul>
    </li>
    '''

def generate_forest_html(forest):
    forest_html = ''.join(generate_html(root) for root in forest)
    return f'''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Collapsible Tree Menu</title>
    <style>
        ul {{
            list-style-type: none; /* Remove bullet points */
        }}
        .nested {{
            display: none;
        }}
        .active {{
            display: block;
        }}
        .caret {{
            cursor: pointer;
            user-select: none;
            font-weight: bold; /* Highlight caret items */
        }}
        .caret::before {{
            content: "\25B6"; /* Right pointing triangle */
            color: black;
            display: inline-block;
            margin-right: 6px;
        }}
        .caret-down::before {{
            content: "\25BC"; /* Down pointing triangle */
        }}
    </style>
</head>
<body>
    <h2>MPILs</h2>
    <ul id="myMenu">
        {forest_html}
    </ul>
    <script>
        document.addEventListener('DOMContentLoaded', function() {{
            var toggler = document.getElementsByClassName("caret");
            for (var i = 0; i < toggler.length; i++) {{
                toggler[i].addEventListener("click", function() {{
                    this.parentElement.querySelector(".nested").classList.toggle("active");
                    this.classList.toggle("caret-down");
                }});
            }}
        }});
    </script>
</body>
</html>
    '''
